# Atención primaria España

In [1]:
import pandas as pd
import os
import json

datos [porta estadístico](https://pestadistico.inteligenciadegestion.mscbs.es/publicoSNS/S/base-de-datos-de-clinicos-de-atencion-primaria-bdcap)

# consulta de datasets

In [2]:
psico_prim = pd.read_excel('../data/at_primaria/detalles_psico.xlsx')

In [3]:
psico_prim.columns = psico_prim.columns.str.replace(" ",'_')

In [4]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Grandes_Grupos,Problema_de_Salud,Casos
157405,2018,02-Aragón,Hombre,18.000-99.999 ? /año,65 y más años,P18 - Abuso de fármacos,NaN
199990,2019,10-Comunidad Valenciana,Hombre,?100.000 ? /año,00-14 años,"P10 - Tartamudeo, disfemia espasmódica, tics",NaN
186554,2019,02-Aragón,Hombre,18.000-99.999 ? /año,15-34 años,P82 - Estrés postraumático,NaN
141413,2017,09-Cataluña,Mujer,Muy baja,00-14 años,P86 - Anorexia nerviosa/bulimia,NaN
4495,2012,04-Baleares,Hombre,18.000-99.999 ? /año,65 y más años,P08 - Ausencia/pérdida de la satisfación sexual,NaN


In [5]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Grandes_Grupos            0
Problema_de_Salud         0
Casos                182993
dtype: int64

In [6]:
psico_prim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213840 entries, 0 to 213839
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Año                213840 non-null  int64  
 1   CCAA               213840 non-null  object 
 2   Sexo               213840 non-null  object 
 3   Nivel_Renta        213840 non-null  object 
 4   Grandes_Grupos     213840 non-null  object 
 5   Problema_de_Salud  213840 non-null  object 
 6   Casos              30847 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 11.4+ MB


In [7]:
psico_prim.columns

Index(['Año', 'CCAA', 'Sexo', 'Nivel_Renta', 'Grandes_Grupos',
       'Problema_de_Salud', 'Casos'],
      dtype='object')

## cambiamos algunos datos categóricos

In [8]:
psico_prim[["CodCA","CCAA"]] = psico_prim.CCAA.str.split("-",expand=True)

In [9]:
# usamos geojson de comunidades para poner el nombre a las comunidades

In [10]:
#diccionario
geo_json = r"../data/socioecon/comunidades.geojson"
with open (geo_json) as geo_file:
    comunidades = json.load(geo_file)

In [11]:
comunidades['features'][0]['properties']

{'cod_ccaa': '07',
 'noml_ccaa': 'COMUNIDAD DE CASTILLA Y LEON',
 'name': 'Castilla-Leon',
 'cartodb_id': 7,
 'created_at': '2014-09-30T00:00:00Z',
 'updated_at': '2014-12-25T02:07:41Z'}

In [12]:
cod_com = {}
for i in range(19):    
    cod_com[comunidades['features'][i]['properties']['cod_ccaa']] = comunidades['features'][i]['properties']['name']

In [13]:
psico_prim['CCAA'] = psico_prim.CodCA.map(cod_com)

In [14]:
list(psico_prim.Nivel_Renta.unique())

['Sin Clasificar',
 '?100.000 ? /año',
 '18.000-99.999 ? /año',
 '<18.000 ? /año',
 'Muy baja']

In [15]:
renta ={'Sin Clasificar':'desconocida',
 '?100.000 ? /año': '+ 100000',
 '18.000-99.999 ? /año': '18000-99999',
 '<18.000 ? /año':'- 18000',
 'Muy baja':'muy_baja'}

In [16]:
psico_prim.Nivel_Renta = psico_prim.Nivel_Renta.replace(renta)

In [17]:
psico_prim.Grandes_Grupos = psico_prim.Grandes_Grupos.str.extract(r'(\d+-*\d+)')
psico_prim.rename({"Grandes_Grupos":"Edad"},axis = 1,inplace = True)
psico_prim.Edad.replace('65','65 +', inplace = True)

In [18]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Problema_de_Salud,Casos,CodCA
150945,2017,Navarra,Hombre,desconocida,15-34,P29 - Otros signos/síntomas psicológicos/mentales,NaN,15
40654,2013,Murcia,Hombre,muy_baja,00-14,P98 - Otras psicosis no especificadas,NaN,14
128846,2017,Aragon,Mujer,+ 100000,15-34,P15 - Abuso crónico del alcohol,NaN,02
77827,2015,Canarias,Hombre,muy_baja,15-34,P79 - Fobia/trastorno compulsivo,NaN,05
34618,2013,Valencia,Mujer,- 18000,15-34,P78 - Neurastenia/surmenage,552.0,10


In [19]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Edad                      0
Problema_de_Salud         0
Casos                182993
CodCA                     0
dtype: int64

In [20]:
psico_prim.Casos.fillna(0.,inplace = True)

In [21]:
psico_prim[["cod_salud","problema_salud"]]= psico_prim.Problema_de_Salud.str.split(" - ",expand=True)

In [22]:
psico_prim.drop('Problema_de_Salud',axis=1,inplace = True)

## equivalencias codigos diagnósticos con CEI10

In [23]:
trans_cod = {'P01': 'F40-49','P02': 'F40-49','P03': 'F40-49','P04': 'F40-49','P05':'F00-F09','P06': 'F30-39','P07': 'F30-39',
'P08': 'F30-39','P09': 'F30-39','P09-07': 'F30-39','P10': 'F90-F98','P11': 'F90-F98','P12': 'F90-F98','P13': 'F90-F98',
'P15': 'F10-F19','P16': 'F10-F19','P17': 'F10-F19','P18': 'F10-F19','P19': 'F10-F19','P20':'F00-F09','P22+23': 'F90-F98',
'P24': 'F80-89','P25': 'F60-69','P27': 'F99','P28': 'F70-79','P29': 'F99','P70':'F00-F09','P71': 'F20-29','P72': 'F20-29',
'P73': 'F20-29','P74': 'F40-49','P75': 'F40-49','P76': 'F30-39','P77': 'F30-39','P78': 'F40-49','P79': 'F40-49','P80': 'F60-69',
'P81': 'F90-F98','P82': 'F40-49','P82/02': 'F40-49','P85': 'F70-79','P86': 'F50-59','P98': 'F20-29','P99': 'F99',
}

In [24]:
psico_prim['cei10'] = psico_prim.cod_salud.map(trans_cod)

In [25]:
diagnos = pd.read_csv('../data/diagnos.csv')

In [26]:
diagnos.sample(2)

,code,diag,cap,title,resumen_cap
9,F09,Trastornos mentales orgánicos o sintomáticos s...,F00-F09,"Trastornos mentales orgánicos, incluidos los t...",orgánicos/sintomáticos
33,F39,"Trastorno del estado de ánimo [afectivo], no e...",F30-39,"Trastornos del estado de animo, trastornos afe...",animo/afectivos


In [27]:
diag_dic = {}
for i,r in diagnos.iterrows():
    diag_dic[r.cap] = r.resumen_cap

In [28]:
psico_prim['resumen'] = psico_prim.cei10.map(diag_dic)

In [29]:
psico_prim.sample()

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,resumen
54594,2014,Castilla-Leon,Mujer,+ 100000,00-14,0.0,07,P78,Neurastenia/surmenage,F40-49,nuróticos/somatomorfos


## tipo columna casos

In [30]:
psico_prim.Casos =psico_prim.Casos.astype('int64')

In [31]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,resumen
35045,2013,Valencia,Mujer,desconocida,65 +,0,10,P24,Problemas específicos del aprendizaje,F80-89,desarrollo psicológico
159762,2018,Asturias,Mujer,+ 100000,35-64,0,03,P98,Otras psicosis no especificadas,F20-29,esquizotípicos/delirantes
182879,2018,Pais Vasco,Mujer,desconocida,65 +,0,16,P16,Abuso agudo del alcohol,F10-F19,consumo psicotrópicos
158882,2018,Asturias,Hombre,+ 100000,00-14,0,03,P98,Otras psicosis no especificadas,F20-29,esquizotípicos/delirantes
94283,2015,Navarra,Hombre,- 18000,35-64,313,15,P79,Fobia/trastorno compulsivo,F40-49,nuróticos/somatomorfos


In [32]:
psico_prim.groupby(['CCAA']).agg({'Año':'min'}).reset_index()
#la Rioja empieza desde el 2016... dropeamos todos estos datos

,CCAA,Año
0,Andalucia,2015
1,Aragon,2011
2,Asturias,2013
3,Baleares,2011
4,Canarias,2011
5,Cantabria,2013
6,Castilla-La Mancha,2012
7,Castilla-Leon,2011
8,Cataluña,2011
9,Extremadura,2013


In [33]:
psico_prim = psico_prim[psico_prim['Año']>=2016]

In [34]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,resumen
120601,2016,Murcia,Mujer,muy_baja,00-14,0,14,P86,Anorexia nerviosa/bulimia,F50-59,factores físicos /fisiológicos
188796,2019,Asturias,Hombre,desconocida,35-64,0,03,P80,Trastornos de la personalidad,F60-69,T. de la personalidad/comportamiento
125359,2016,La Rioja,Mujer,+ 100000,35-64,0,17,P04,Sensación/sentimientos de irritabilidad/enojo,F40-49,nuróticos/somatomorfos
141451,2017,Cataluña,Mujer,muy_baja,15-34,0,09,P79,Fobia/trastorno compulsivo,F40-49,nuróticos/somatomorfos
151828,2017,Navarra,Mujer,desconocida,15-34,0,15,P72,Esquizofrenia,F20-29,esquizotípicos/delirantes


In [35]:
psico_prim.to_csv('../data/psico_atprimaria.csv',index=False)